<h3>Process Online Survey Responses

In [49]:
import os

# make dict with ex_123.mp3 as key, and list of values made from real filename
# values list: [phrase_int, target_emotion_int, cond/nocond/orig]
# can make a dict from a list of list pairs or tuple pairs i.e. could use zip to zip two lists, then use output to 
# make dict with one list as keys and other as values 
    # dict = {'Name': 'Zara', 'Age': 7, 'Class': 'First'}
    # print "dict['Name']: ", dict['Name']
    # print "dict['Age']: ", dict['Age']

# example of what I want
#                              p  e  c
# dict_files = {'ex_101.mp3': [1, 2, 1],
#               'ex_102.mp3': [3, 1, 2],
#               'ex_103.mp3': [5, 2, 1]}

# load rename_log.txt
input_directory_path = '/Users/robinson/Downloads/data/pred/z_evaluation_final/renamed_normed_numbered/'
filename_in = 'rename_log.txt'
fi = open(os.path.join(input_directory_path, filename_in), 'r')
# read the lines in
fi_lines = fi.read().split('\n')
# print(fi_lines)

# create empty dict
dict_files = {}

# for each line in the file
for line in fi_lines:
    
    # series of if/else for phrase, emotion, cond (11 total)
    if 'p01' in line:
        phrase = 1
    elif 'p03' in line:
        phrase = 3
    elif 'p05' in line:
        phrase = 5
    elif 'p10' in line:
        phrase = 10
    else:
        continue
    
    # make emotions 1,2,3,4 to match survey output 
    if 'orig' in line:
        filetype = 3
        if 'e02' in line:
            emotion = 1
        elif 'e04' in line:
            emotion = 2
        elif 'e06' in line:
            emotion = 3
        elif 'e08' in line:
            emotion = 4
        else:
            continue        
    elif 'nocond' in line:
        filetype = 2
        if 'JOY_2' in line:
            emotion = 1
        elif 'PEUR_4' in line:
            emotion = 2
        elif 'TRI_6' in line:
            emotion = 3
        elif 'COL_8' in line:
            emotion = 4
        else:
            continue        
    elif 'cond' in line:
        filetype = 1
        if 'JOY_2' in line:
            emotion = 1
        elif 'PEUR_4' in line:
            emotion = 2
        elif 'TRI_6' in line:
            emotion = 3
        elif 'COL_8' in line:
            emotion = 4
        else:
            continue        
    else:
        continue

    # split line by comma, take second part
    wav_filename = line.split(',')[1]
    # split part with split_ext, take first part
    wav_filename_base = wav_filename.split('.')[0]
    # set mp3_filename to base+.mp3
    mp3_filename = wav_filename_base + '.mp3'
    # add entry to dict e.g. dict_files['ex_101.mp3'] = [1, 2, 1]
    dict_files[mp3_filename] = [phrase, emotion, filetype]
    
    # print(line)
    # print(dict_files[mp3_filename])

# print(dict_files)
# close fi
fi.close()

In [50]:
import csv

# NOTE: uses dict_files from previous step

# input directory path where all the response text files are stored
input_directory_path = '/Users/robinson/Dropbox/anasynth/_web/responses'

# define output directory and files
output_directory_path = os.path.join(input_directory_path, 'out')
if not os.path.exists(output_directory_path):
    os.mkdir(output_directory_path)
# create output file to write to
filename_out = 'output.txt'
fo = open(os.path.join(output_directory_path, filename_out), 'w')

# make csv writer to write the file
csvWriterS = csv.writer(fo)  
# write header row
csvWriterS.writerow(('file', 'phrase', 'emo', 'ftype', 'resp_emo', 'resp_qual', 
                    'native', 'equip'))


# for each file in the directory
for file in os.listdir(input_directory_path):
    # get filename
    filename = os.fsdecode(file)
    if filename.endswith('.txt'):
    
        # open file and split into list of lines
        fi = open(os.path.join(input_directory_path, filename), 'r')
        # read the lines in
        fi_lines = fi.read().split('\n')
    
        # for each line, starting at the end and working upwards
        # for each line in the file
        for line in reversed(fi_lines):
            # if line starts with info_comment, put the full line into comment_text var
            if 'info_comment' in line:
                comment_text = line
            
            # if line starts with info_environ, get the last char and put in equipment_device var
            elif 'info_environ' in line:
                equipment_device = line
            
            # if line starts with info_experience
            #    get the last char and put in native_french var
            elif 'info_experience' in line:
                native_french = line
                
            # if line starts with ex, get values and write out, including using previously set variables
            elif 'ex_' in line:
                # split it by :  (line.split(':')])
                splitline = line.split(':')
                # put item 1 in mp3_filename var
                mp3_filename = splitline[0]
                # put item 2 in emotion_reponse var
                emotion_reponse = splitline[1]
                # put item 3 in quality_reponse var
                quality_reponse = splitline[2]
                
                # print('emotion_reponse OLD ', emotion_reponse)
                # translate from (dodgy) website emotion values to proper values
                # happiness: website 1, real value 1
                if emotion_reponse == '1':
                    emotion_reponse = '1'
                # fear: website 4, real value 2
                elif emotion_reponse == '4':
                    emotion_reponse = '2'
                # sadness: website 2, real value 3
                elif emotion_reponse == '2':
                    emotion_reponse = '3'
                # anger: website 3, real value 4
                elif emotion_reponse == '3':
                    emotion_reponse = '4'
                # print('emotion_reponse NEW ', emotion_reponse)
            
                # look up mp3_filename and get list of file attribute values
                fa = dict_files[mp3_filename + '.mp3']
                
                # write out CSV with format:
                # mp3_filename, phrase_int, target_emotion_int, cond/nocond/orig, emotion_reponse, quality_reponse, native_french, equipment_device 
                csvWriterS.writerow((mp3_filename, fa[0], fa[1], fa[2], emotion_reponse, quality_reponse, 
                                    native_french[-1], equipment_device[-1]))
                
# close fo
fo.close()
print('done')



done
